# Simple RNN

Intended as a baseline for the Transformer. Based on the original classifier from RAPID, Muthukrishna et al (2019)

Author: Kara Ponder (SLAC)

In [ ]:
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
%matplotlib inline

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.layers import Dense, Input, Embedding
from tensorflow.python.keras.layers import LSTM, GRU
from tensorflow.python.keras.layers import Dropout, BatchNormalization, Activation, TimeDistributed, Masking
from tensorflow.python.keras.layers.convolutional import Conv1D, Conv2D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D, MaxPooling2D

In [ ]:
#N = 100 # number of objects
N_days = 50 #101
Nf = 6 # number of filters

batch_size=64

This is how you can load in PLAsTiCC data

In [ ]:
num_class = 8

data_dir = '/scratch/[YOUR INFO]'
data_extension = '_minmax_yspline_yt0_multiclass_maybefullci()_zNone_bTrue_ig(88,92,65,16,53,6).npy'

# Read in dataset
_lc_data = np.load(data_dir + 'X_train' + data_extension)
_wgt_map = np.load(data_dir + 'X_wgtmap_train' + data_extension)
_pre_mask_map = np.ma.masked_values(_lc_data, 0)
_mask_map = np.ones(np.shape(_lc_data))
_mask_map[_pre_mask_map.mask] = 0.0

label = np.load(data_dir + 'y_train' + data_extension)

Or you can instead load in the test data in the repository

In [ ]:
num_class = 4

label = np.load('label.npy')
lc_data = np.load('lc_data.npy')

labels = tf.keras.utils.to_categorical(label, num_classes=num_class, dtype="float64")

dataset = tf.data.Dataset.from_tensor_slices((_lc_data, label[:, -1, :])) #(lc_data, labels))
batch_ds = dataset.batch(batch_size)

Define the model using tf.Sequential.
Compile.

In [ ]:
n_timesteps, n_features, n_outputs = N_days, Nf, num_class

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features))) #, return_sequences=True
model.add(Dropout(0.0))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(0.00001)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Fit for the model weights. 

In [ ]:
history = model.fit(batch_ds, epochs=10)